In [1]:
from policyengine_us.data.datasets.cps.raw_cps import RawCPS_2018, RawCPS_2019

In [2]:
RawCPS_2019()
RawCPS_2018()

Downloaded ASEC: 100%|██████████| 162M/162M [00:58<00:00, 2.78MiB/s]  

In [ ]:
cps_current_year_data = RawCPS_2019().load()
cps_previous_year_data = RawCPS_2018().load()
cps_previous_year = cps_previous_year_data.person.set_index(
    cps_previous_year_data.person.PERIDNUM
)
cps_current_year = cps_current_year_data.person.set_index(
    cps_current_year_data.person.PERIDNUM
)

previous_year_data = cps_previous_year[["WSAL_VAL", "SEMP_VAL"]].rename(
    {
        "WSAL_VAL": "employment_income_last_year",
        "SEMP_VAL": "self_employment_income_last_year",
    },
    axis=1,
)

joined_data = cps_current_year.join(previous_year_data)[
    ["employment_income_last_year", "self_employment_income_last_year"]
]
joined_data["previous_year_income_available"] = (
    ~joined_data.employment_income_last_year.isna()
    & ~joined_data.self_employment_income_last_year.isna()
)
joined_data = joined_data.fillna(-1)

import pandas as pd

cps = pd.DataFrame()

cps["employment_income"] = cps_current_year.WSAL_VAL.values
cps["self_employment_income"] = cps_current_year.SEMP_VAL.values

# CPS already ordered by PERIDNUM, so the join wouldn't change the order.
cps["employment_income_last_year"] = joined_data[
    "employment_income_last_year"
].values
cps["self_employment_income_last_year"] = joined_data[
    "self_employment_income_last_year"
].values
cps["previous_year_income_available"] = joined_data[
    "previous_year_income_available"
].values